# Class III - An introduction to PowerModels.jl

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of the [PowerModels.jl](https://github.com/lanl-ansi/PowerModels.jl) package. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) and [Class II - An introduction  to JuMP](Class%20II%20-%20An%20introduction%20to%20JuMP.ipynb) first.

As in Class II, run the following magic sauce to check we're good to go.

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

### Background 

This [presentation](https://github.com/lanl-ansi/tutorial-grid-science-2019/blob/master/assets/infrastructure_optimization_in_julia.pdf) provides motivation and context for this notebook.

Some additional informaiton is also available at,
* [PowerModels Documentation](https://lanl-ansi.github.io/PowerModels.jl/stable/)
* [PowerModels Network Model Update](https://youtu.be/j7r4onyiNRQ)
* [PSCC 2018 Presentation](https://youtu.be/AEEzt3IjLaM)
* [JuMP Dev 2017 Presentation](https://youtu.be/W4LOKR7B4ts)


### Working with the Network Model

A simple 5-bus network model is provided with this tutorial.  The matpower file can be viewed [here](../edit/data/pglib_opf_case5_pjm.m).

The `parse_file` function is used to load a text files into the PowerModels data model, 

In [ ]:
using PowerModels

data = PowerModels.parse_file("data/pglib_opf_case5_pjm.m")

In this case the file parser generated a few informational of messages.  

The first `info` message indicates that the file contains a data block named `area` that is not part of the Matpower data specification.  Still the parser will load this data using some standards for [Matpower data extentions](https://lanl-ansi.github.io/PowerModels.jl/stable/network-data/#Working-with-Matpower-Data-Files-1).

The second block of `warn` messages indicates that PowerModels has detected that the generator cost model could be simplified.  In this case a linear function specified as a quadratic function was transformed into a linear function.  PowerModels makes a best effort to clean data while parsing, but the best practice is to clean datasets until they do not produce any warning messages in PowerModels.

Let us use the silence command to ignore similar warnings for the rest of this tutorial.

In [ ]:
PowerModels.silence()

Now, lets look at some of the network data.  The data for the first bus, load and branch can be viewed as follows,

In [ ]:
data["bus"]["1"]

In [ ]:
data["load"]["1"]

Notice that the "pd" value is 3.0, while in the Matpower file it was 300.  By default all data in InfrastructureModels is in per-unit.  The functions `make_mixed_units` and `make_per_unit` can be used for switching back and forth.

In [ ]:
data["branch"]["1"]

That's great, but looking at components one-by-one can get boring fast.

All InfrastructureModels packages provide a `print_summary` function that prints a table-like summary of the network data to the terminal.

In [ ]:
PowerModels.print_summary(data)

Dictionary comprehensions also provide an easy way to take slices of the data.

For example lets inspect the generation profile and output capabilities using comprehensions.

In [ ]:
[gen["pg"] for (i,gen) in data["gen"]]

In [ ]:
[gen["pg"] + gen["qg"]im for (i,gen) in data["gen"]]

In [ ]:
[(gen["qmin"], gen["qmax"]) for (i,gen) in data["gen"]]

### Solving Optimal Power Flow (OPF)

Before we can solve a power model, we need a solver.  Lets start with the simple linear programming solver GLPK and try to solve a DC OPF model.

In [ ]:
using GLPK
lp_solver = GLPK.Optimizer

result = run_dc_opf(data, lp_solver)

The result object contains a variety of useful information about the optimization problem solved, including the objective value and wall clock runtime.  Detailed documentation of the result dictionary is available [here](https://lanl-ansi.github.io/PowerModels.jl/stable/result-data/), however most interesting point is the `solution` data.

In [ ]:
result["solution"]

Not every useful, lets try `print_summary`,

In [ ]:
PowerModels.print_summary(result["solution"])

Looks great!  But `vm` and `qg` are missing...  On to AC OPF!

In [ ]:
result = run_ac_opf(data, lp_solver)

#### Quiz Question
That was ugly... What happened?!?


Lets try using an NLP solver.

In [ ]:
using Ipopt
nlp_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

result = run_ac_opf(data, nlp_solver)

It worked!  Lets look at the solution,

In [ ]:
PowerModels.print_summary(result["solution"])

The solution make sense intuitively.  The voltage magnitude on bus 3 is pushed to the upper limit, presumably to minimize line losses.  Total generation costs are similar to DC-OPF which seems reasonable.

The solution seems reasonable, but is it optimal?  To test that we need a convex relaxation!  But first a brief digression.

A little secret, the functions `run_dc_opf` and `run_ac_opf` are short hands for a much more general `run_opf` function which supports many variants of optimal power flow problem.  Specifically,
```
run_dc_opf(data, solver) => run_opf(data, DCPPowerModel, solver)
run_dc_opf(data, solver) => run_opf(data, ACPPowerModel, solver)
```

Try the following code with different PowerModels.  Suggested models include `ACPPowerModel`,`ACRPowerModel`,`DCPPowerModel`,`DCPLLPowerModel`,`LPACCPowerModel`, `SOCWRPowerModel`, `QCWRPowerModel`.  A fairly complete list of implemented models is available [here](https://lanl-ansi.github.io/PowerModels.jl/stable/formulation-details/).

In [ ]:
result = run_opf(data, ACPPowerModel, nlp_solver)
PowerModels.print_summary(result["solution"])
println()
println("Objective: $(result["objective"])")

At this point you might be curious what is PowerModels doing under the hood.  In short, it builds and solves JuMP models.  The [PowerModelsAnnex](https://github.com/lanl-ansi/PowerModelsAnnex.jl/tree/master/src/model) includes some examples of the kind of JuMP models that PowerModels when you call `run_opf`.

Another way to see what PowerModels is doing is to pass your own JuMP model, then you can inspect it later.

In [ ]:
using JuMP

m = Model()
result = run_opf(data, DCPPowerModel, nlp_solver, jump_model=m)

println(m)

### Computing Optimality Gaps

Getting back to the question of AC-OPF solution quality.  An easy way to certify AC-OPF solution quality is to use a convex-relaxation to compute a lower bound.  If the "gap" between the solution and the lower bound is small we are happy.

Let build a helper function for computing optimality gaps.

In [ ]:
function soc_gap(data_file, nlp_solver)
    data = PowerModels.parse_file(data_file)
    ac_result = run_opf(data, ACPPowerModel, nlp_solver)
    soc_result = run_opf(data, SOCWRPowerModel, nlp_solver)
    
    # it is important to check the solver terminated nominally!
    @assert  ac_result["termination_status"] == OPTIMAL ||  ac_result["termination_status"] == LOCALLY_SOLVED
    @assert soc_result["termination_status"] == OPTIMAL || soc_result["termination_status"] == LOCALLY_SOLVED
    
    # by convention optimality gaps are given as a percentage
    gap = 100*(ac_result["objective"] - soc_result["objective"])/ac_result["objective"]
    
    return (gap=gap, ub_time=ac_result["solve_time"], lb_time=soc_result["solve_time"]) 
end

In [ ]:
soc_gap("data/pglib_opf_case5_pjm.m", nlp_solver)

A 14% gap, that's HUGE!  It is not clear if the AC-OPF solutoin is optimal or not.  Clearly this case requires more study.

In the mean time, try the `soc_gap` function on a few more of the network cases included with this tutorial.  Available datasets are `pglib_opf_case14_ieee.m`, `pglib_opf_case30_ieee.m`, `pglib_opf_case73_ieee_rts.m`, `pglib_opf_case118_ieee.m`, `pglib_opf_case200_tamu.m`, `pglib_opf_case500_tamu.m`, `pglib_opf_case1354_pegase.m`, `pglib_opf_case2383wp_k.m` (larger may take a few minutes to run).  The complete PGLib-OPF benchmark library with over 100 cases is available [here](https://github.com/power-grid-lib/pglib-opf).

In [ ]:
soc_gap("data/pglib_opf_case14_ieee.m", nlp_solver)

#### Discussion Point

Can you draw any conclusions from computing optimality gaps on a variety of cases?


### A Simple Contingency Study

Component `status` parameters can be useful for exploring a variety of component failure scenarios.  Lets use it to do a simple generator N-1 study on the 5-bus example.


In [ ]:
function single_generator_loss(data_file, nlp_solver)
    data = PowerModels.parse_file(data_file)
    
    result = run_opf(data, ACPPowerModel, nlp_solver)

    # it is important to check the solver terminated nominally!
    @assert  result["termination_status"] == OPTIMAL || result["termination_status"] == LOCALLY_SOLVED

    println("base cost: $(result["objective"])")
    
    for (i,gen) in sort(data["gen"])
        # remove the generator
        gen["gen_status"] = 0
        
        result = run_opf(data, ACPPowerModel, nlp_solver)
        
        println("gen $(i) out: $(result["objective"]), $(result["termination_status"])")

        # restore the generator
        gen["gen_status"] = 1
    end
end

In [ ]:
single_generator_loss("data/pglib_opf_case5_pjm.m", nlp_solver)

This simple study seems to suggest that this network is not N-1 secure and cannot support the loss of generator 3 or 5.  Let's compute how much these two generators contribute to the total generation capacity in the data set.

In [ ]:
pg_capacity = sum(gen["pmax"] for (i,gen) in data["gen"])

In [ ]:
100*data["gen"]["3"]["pmax"]/pg_capacity

In [ ]:
100*data["gen"]["5"]["pmax"]/pg_capacity

It seems reasonable that losing over 30% of the total generating capacity would be problematic.

### Solving Optimal Transmission Switching (OTS)

Because InfrastructureModels is built on JuMP, is it fairly easy to encode challenging mixed integer nonlinear optimization problems.  Solving these problems however presents additional challenges.

Lets consider the OTS problem.  An extension of OPF where branches can be removed from the network (i.e. a binary decion).  Lets try solving OPF and OTS on the 5-bus example.  Because the OTS problem has discrete variables we will need to use a MIP solver for that one.

In [ ]:
data = PowerModels.parse_file("data/pglib_opf_case5_pjm.m")

opf_result = run_opf(data, DCPPowerModel, lp_solver)

In [ ]:
using Cbc

mip_solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 0)

ots_result = run_ots(data, DCPPowerModel, mip_solver)

Look at those massive cost savings!!!  Lets look at the solution to see what happened.

In [ ]:
PowerModels.print_summary(ots_result["solution"])

It seems that removing branch 5 from the network will remove congestion caused by Ohm's Law and yield the observed savings.

#### OTS Home Work

Explore the AC-OTS problem on the 5-bus case.  Build the model with commands like `run_ots(data, ACPPowerModel, <solver>)` and `run_ots(data, SOCWRPowerModel, <solver>)`.  Use solvers like Juniper/Bonmin as heuristics to find upper bound solutions and solvers like Juniper/Pavito to find lower bounds.  Compare the gaps similar to how we did for the OPF probem.

### Other Topics

PowerModels includes a variety of useful features that are outside the scope of this brief tutorial.  These features include,
* Extracting Duals/LMPs from linear models using output options
* Network models with HVDC lines or storage devices
* Optimization Based Bound Tightening algorithms
* Using mulitnetworks for time series analysis
* Using multiconductors to model three phase networks (e.g. see [ThreePhasePowerModels](https://github.com/lanl-ansi/ThreePhasePowerModels.jl))
* A variety of experimental features in [PowerModelsAnnex](https://github.com/lanl-ansi/PowerModelsAnnex.jl)